In [ ]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    get_region_mapping_base,
)
from polars import testing

In [ ]:
region.Federal2022

In [ ]:
region.SA1_2021

In [ ]:
region.Federal2022.geometry

In [ ]:
region.SA1_2021.geometry.head(100)

In [ ]:
from electoralyze.region.redistribute.mapping import _get_intersection_area

In [ ]:
geometry_from = region.Federal2022.geometry
geometry_to = region.SA1_2021.geometry.head(1000)

In [ ]:
geometry_combined = geometry_from.rename({"geometry": "geometry_from"}).join(
    geometry_to.rename({"geometry": "geometry_to"}), how="cross"
)
intersection_area = geometry_combined.select(
    pl.exclude("geometry_from", "geometry_to"),
    st.geom("geometry_from").st.intersection(st.geom("geometry_to")).st.area().alias("intersection_area"),
)
intersection_area

In [ ]:
geometry_combined = (
    geometry_from.pipe(to_geopandas)
    .rename(columns={"geometry": "geometry_from"})
    .merge(geometry_to.pipe(to_geopandas).rename(columns={"geometry": "geometry_to"}), how="cross")
)
geometry_combined

In [ ]:
intersection_area = (
    geometry_combined
    # .loc[lambda df: df["geometry_from"].intersects(df["geometry_to"])] # THIS TAKES MUCH LONGER (3x), REMOVE
    .assign(intersection_area=lambda df: df["geometry_from"].intersection(df["geometry_to"]).area).drop(
        ["geometry_from", "geometry_to"], axis=1
    )
)
intersection_area

In [ ]:
_get_intersection_area(
    geometry_from=region.Federal2022.geometry,
    geometry_to=region.SA1_2021.geometry.head(100),
).pipe(print)

In [ ]:
(
    region.Federal2022.geometry.join(
        region.SA1_2021.geometry.head(10000),
        how="cross",
    )
    .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
)

In [ ]:
(
    region.Federal2022.geometry.pipe(to_geopandas)
    .merge(
        region.SA1_2021.geometry.head(10000).pipe(to_geopandas),
        how="cross",
    )
    # .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    # .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
    .loc[lambda df: df["geometry_x"].overlaps(df["geometry_y"])]
    .assign(area=lambda df: df["geometry_x"].intersection(df["geometry_y"]).area)
)

In [ ]:
(
    region.Federal2022.geometry.pipe(to_geopandas)
    .merge(
        region.SA1_2021.geometry.head(100).pipe(to_geopandas),
        how="cross",
    )
    # .filter(st.geom("geometry").st.overlaps(pl.col("geometry_right")))
    # .with_columns(st.geom("geometry").st.intersection(pl.col("geometry_right")).st.area())
    .loc[lambda df: df["geometry_x"].overlaps(df["geometry_y"])]
    .assign(area=lambda df: df["geometry_x"].intersection(df["geometry_y"]).area)
)

In [ ]:
some_intersection = _create_intersection_area_mapping(
    geometry_from=region.Federal2022.geometry, geometry_to=region.SA1_2021.geometry.head(100)
)
some_intersection

In [ ]:
target_sa1 = 10103101401
target_sa1 = 10106154239
region.SA1_2021.geometry.filter(pl.col(region.SA1_2021.id) == target_sa1).select(st.geom("geometry").st.area())

In [ ]:
(
    region.SA1_2021.geometry.filter(pl.col(region.SA1_2021.id) == target_sa1)
    .join(
        region.Federal2022.geometry.filter(pl.col(region.Federal2022.id).is_in(["Eden-Monaro", "Hume"])),
        how="cross",
    )
    .select(st.geom("geometry").st.intersection(pl.col("geometry_right")))
    # .head(1)
    # .tail(1)
    .pipe(to_geopandas)
    .explore()
)

In [ ]:
some_intersection.filter(pl.col(region.SA1_2021.id) == target_sa1)

In [ ]:
_create_intersection_area_mapping(
    geometry_from=region.Federal2022.get_raw_geometry(), geometry_to=region.SA1_2021.get_raw_geometry().head(100)
)

In [ ]:
intersection = get_region_mapping_base(
    region_from=region.Federal2022,
    region_to=region.SA1_2021,
    mapping_method="intersection_area",
    redistribute_with_full=True,
    save_data=True,
    force_new=True,
)
intersection

In [ ]:
intersection.group_by("SA1_2021").agg(pl.col("federal_2022").count().alias("count")).sort("count")

In [ ]:
lots_of_federals_sa1 = 40602114110
intersection.filter(pl.col("SA1_2021") == lots_of_federals_sa1)

In [ ]:
intersection.filter(pl.col("SA1_2021").is_null()).sum().pipe(print)
intersection.filter(pl.col("SA1_2021").is_null()).sort("mapping")

In [ ]:
specified_sa1 = intersection.filter(pl.col("federal_2022").is_in(["Leichhardt"]))[region.SA1_2021.id].unique().to_list()

intersection.filter(pl.col("federal_2022").is_in(["Leichhardt"]))

In [ ]:
region.SA1_2021.geometry.filter(pl.col(region.SA1_2021.id).is_in(specified_sa1)).pipe(to_geopandas).explore()

In [ ]:
region.Federal2022.geometry.filter(pl.col(region.Federal2022.id).is_in(["Leichhardt"])).pipe(to_geopandas).explore()

In [ ]:
intersection.filter(pl.col("federal_2022").is_null()).sum()

In [ ]:
intersection["mapping"].sum()